### 1. Install Packages

In [1]:
!pip install flask
!pip install flask-ngrok
print("Installed")

Installed


In [2]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install kss
!pip install okt
!pip install konlpy
!pip install gluonnlp
!pip install mxnet
!pip install sentence-transformers

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4d5qvwuy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4d5qvwuy
     |████████████████████████████████| 132 kB 12.0 MB/s 
     |████████████████████████████████| 344 kB 49.1 MB/s 
     |████████████████████████████████| 47.3 MB 1.5 MB/s 
     |████████████████████████████████| 4.5 MB 40.7 MB/s 
     |████████████████████████████████| 1.2 MB 35.5 MB/s 
     |████████████████████████████████| 4.0 MB 35.7 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 880 kB 39.2 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 8.7 MB 37.4 MB/s 
     |████████████████████████████████| 127 kB 50.9 MB/s 
  Created wheel for kobert: filename=ko

### 2. Install Ngrok

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

--2022-05-11 11:27:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  14.5MB/s    in 0.9s    

2022-05-11 11:27:28 (14.5 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok


### 3. Ensure Ngrok is Installed

In [4]:
!./ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

### 4. Set Ngrok Token

In [5]:
!./ngrok authtoken 28vL9ko9ifzxsPY0BNrduIR4Wkw_3qAy4kCWkbzB9HEaz3VrK

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


### 5.Building Flask App

In [17]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('/content/drive/MyDrive/Backend_ML') #/content/drive/MyDrive/'your path'
for i in sys.path:
    print(i)

import os
print(os.getcwd())


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).

/content
/env/python
/usr/lib/python37.zip
/usr/lib/python3.7
/usr/lib/python3.7/lib-dynload
/usr/local/lib/python3.7/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.7/dist-packages/IPython/extensions
/root/.ipython
/content/drive/MyDrive/Backend_ML
/content/drive/MyDrive/Backend_ML
/content/drive/MyDrive/Backend_ML
/content/drive/MyDrive/Backend_ML
/content/drive/MyDrive/Backend_ML
/content


In [ ]:
from flask import Flask,request, jsonify, abort, make_response
from flask_ngrok import run_with_ngrok
from sentiment.sentiment_model import get_sentiment_model_class
from keyword_extractor.keyword_model import get_keyword_model_class
import yaml

app = Flask(__name__)
run_with_ngrok(app)

with open('/content/drive/MyDrive/Backend_ML/sentiment/configs/cfg.yaml') as f:
    sentiment_cfg = yaml.load(f, Loader=yaml.FullLoader)
with open('/content/drive/MyDrive/Backend_ML/keyword_extractor/configs/cfg.yaml') as f:
    keyword_cfg = yaml.load(f, Loader=yaml.FullLoader)
    
sentiment_model_class = get_sentiment_model_class(sentiment_cfg['model_name'])
sentiment_model = sentiment_model_class(sentiment_cfg)

keyword_model_class = get_keyword_model_class(keyword_cfg['model_name'])
keyword_model = keyword_model_class(keyword_cfg)

# 서버 작동 여부 확인 Route ('/')
@app.route('/')
def server_check():
    return jsonify(message="Server Linked!")

# Emotion Prediction Route ('/prediction')
@app.route('/prediction', methods = ['POST'])
def prediction():
    # request를 json으로 받아온다.
    req = request.get_json()
    content = req['content']
    
    emotion = sentiment_model.predict(content)
    hashtag = keyword_model.predict(content)

    return jsonify(emotion= emotion,hashtag_1 = hashtag[0],hashtag_2 = hashtag[1],hashtag_3 = hashtag[2])

@app.errorhandler(500)
def internal_server_error(e):
    abort(make_response(jsonify(message = "Internal Server Error"),500))

if __name__ == '__main__':
    app.run()
    app.register_error_handler(500,internal_server_error)



using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[Korean Sentence Splitter]:  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2e92-35-240-37-23.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:48:14] "POST /prediction HTTP/1.1" 200 -


[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:49:04] "POST /prediction HTTP/1.1" 200 -


...

[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:51:22] "POST /prediction HTTP/1.1" 200 -


[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:51:44] "POST /prediction HTTP/1.1" 200 -


[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:52:30] "POST /prediction HTTP/1.1" 200 -


[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:52:44] "POST /prediction HTTP/1.1" 200 -


[Korean Sentence Splitter]: 127.0.0.1 - - [11/May/2022 12:53:44] "POST /prediction HTTP/1.1" 200 -
